In [2]:
import gzip
import itertools
import string
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

import warnings

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

%matplotlib inline

In [32]:
import json
with open('../data/Amazon_Instant_Video_5.json') as f:
    video = pd.DataFrame(json.loads(line) for line in f)

In [33]:
video.describe()

,overall,unixReviewTime
count,37126.00000,3.712600e+04
mean,4.20953,1.376795e+09
std,1.11855,3.054958e+07
min,1.00000,9.754560e+08
25%,4.00000,1.367798e+09
50%,5.00000,1.384992e+09
75%,5.00000,1.394150e+09
max,5.00000,1.406074e+09


In [34]:
video.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B000H00VBQ,"[0, 0]",2.0,I had big expectations because I love English ...,"05 3, 2014",A11N155CW1UV02,AdrianaM,A little bit boring for me,1399075200
1,B000H00VBQ,"[0, 0]",5.0,I highly recommend this series. It is a must f...,"09 3, 2012",A3BC8O2KCL29V2,Carol T,Excellent Grown Up TV,1346630400
2,B000H00VBQ,"[0, 1]",1.0,This one is a real snoozer. Don't believe anyt...,"10 16, 2013",A60D5HQFOTSOM,"Daniel Cooper ""dancoopermedia""",Way too boring for me,1381881600
3,B000H00VBQ,"[0, 0]",4.0,Mysteries are interesting. The tension betwee...,"10 30, 2013",A1RJPIGRSNX4PW,"J. Kaplan ""JJ""",Robson Green is mesmerizing,1383091200
4,B000H00VBQ,"[1, 1]",5.0,"This show always is excellent, as far as briti...","02 11, 2009",A16XRPF40679KG,Michael Dobey,Robson green and great writing,1234310400


In [35]:
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(video.reviewText)
X_counts.shape

(37126, 54045)

In [36]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)

X_tfidf.shape

(37126, 54045)

* I consider reviews greater than 3 to be positive.

In [37]:
y = [1 if x > 3 else 0 for x in video.overall]

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, random_state=42)

In [39]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)
predicted = clf.predict(X_test)
np.mean(predicted == y_test)

0.7932557638439991

* 79% accuracy is pretty good.

* Let's see if we can improve with another method

In [40]:
from sklearn.linear_model import SGDClassifier

svmlin = svm.SVC()
svmlin.fit(X_train, y_train)

predict = svmlin.predict(X_test)
np.mean(predict == y_test)

0.7926093514328808

* Doesn't quite work out as expected

* Let's see how our model works on another group like Lawn and Patio furniture

* We call transform instead of fit_transform to create our td_idf features.


In [41]:
with open('../data/Patio_Lawn_and_Garden_5.json') as f:
    patio = pd.DataFrame(json.loads(line) for line in f)

In [42]:
patioy = [1 if x > 3 else 0 for x in patio.overall]

patioX_counts = count_vect.transform(patio.reviewText)
patioX_counts.shape

(13272, 54045)

In [43]:
#tfidf_transformer = TfidfTransformer()
patioX_tfidf = tfidf_transformer.transform(patioX_counts)

patioX_tfidf.shape

(13272, 54045)

In [45]:
predicted_patio = clf.predict(patioX_tfidf)
np.mean(predicted_patio == patioy)

0.7851868595539482

* This looks surprisingly good for a completely different set of items.

* Video is quite different than Lawn furniture.

* However, our tf_idf features appear to be rather robust.